In [ ]:
import os
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nengo
from scipy import stats
from nengo.utils.matplotlib import rasterplot
from nengo.utils.stdlib import Timer

import phd

# Some plotting niceties
plt.rc('figure', figsize=(10, 6))
sns.set_style('white')
sns.set_style('ticks')

def img(array, zscore=False):
    if zscore:
        array = stats.zscore(array, axis=0)
    plt.pcolormesh(array.T)
    plt.ylim(top=array.shape[1])
    plt.xlim(right=array.shape[0])
    plt.yticks(())
    sns.despine(left=True)
    plt.tight_layout()

def img_compare(array1, array2, zscore=(False, False)):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    img(array1, zscore[0])
    plt.subplot(1, 2, 2)
    img(array2, zscore[1])
    plt.tight_layout()

timit_path = "~/phd_data/timit"
timit = phd.timit.TIMIT(timit_path)
try:
    timit.untar(os.path.expanduser("~/Dropbox/LDC93S1.tgz"))
except Exception as e:
    pass

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

## Basic usage

In [ ]:
timit = phd.timit.TIMIT(timit_path)
timit.filefilt.spkr_id = "CAG0"
samples = timit.word_samples(['wash'])
model = phd.sermo.AuditoryFeatures()
model.audio = samples['wash'][0]

In [ ]:
# Get MFCCs
model.mfcc.dt = 0.01
x = model.mfcc()
print x.shape
img_compare(x, x, zscore=(False, True))

In [ ]:
# Get NCCs
net = model.build()
with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.output, synapse=0.01)
sim = nengo.Simulator(net, dt=0.001)
sim.run(model.t_audio)

img_compare(sim.data[c_p], sim.data[c_p], zscore=(False, True))

In [ ]:
# A peek into the neural implementation
plt.figure(figsize=(8, 14))
plt.subplot(3, 1, 1)
phd.plots.cochleogram(sim.data[ihc_p], sim.trange(), model.freqs)
plt.subplot(3, 1, 2)
phd.plots.cochleogram(sim.data[an_in_p], sim.trange(), model.freqs)
plt.subplot(3, 1, 3)
rasterplot(sim.trange(), sim.data[an_p])
plt.ylim(0, net.periphery.an.n_neurons * net.periphery.an.n_ensembles)
sns.despine()
plt.tight_layout()

## With derivatives

In [ ]:
timit = phd.timit.TIMIT(timit_path)
timit.filefilt.spkr_id = "CAG0"
samples = timit.word_samples(['wash'])
model = phd.sermo.AuditoryFeatures()
model.audio = samples['wash'][0]
model.add_derivative()  # First derivative
model.add_derivative()  # Second derivative

In [ ]:
# Get MFCCs
model.mfcc.dt = 0.01
x = model.mfcc()
print x.shape
img_compare(x, x, zscore=(False, True))

In [ ]:
# Get NCCs
net = model.build()
with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.output, synapse=0.01)
sim = nengo.Simulator(net, dt=0.001)
sim.run(model.t_audio)

img_compare(sim.data[c_p], sim.data[c_p], zscore=(False, True))

## With Adaptive LIF neurons in AN

In [ ]:
timit = phd.timit.TIMIT(timit_path)
timit.filefilt.spkr_id = "CAG0"
samples = timit.word_samples(['wash'])
model = phd.sermo.AuditoryFeatures()
model.audio = samples['wash'][0]
model.periphery.adaptive_neurons = True

In [ ]:
# Get MFCCs
model.mfcc.dt = 0.01
x = model.mfcc()
print x.shape
img_compare(x, x, zscore=(False, True))

In [ ]:
# Get NCCs
net = model.build()
with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.output, synapse=0.01)
sim = nengo.Simulator(net, dt=0.001)
sim.run(model.t_audio)

img_compare(sim.data[c_p], sim.data[c_p], zscore=(False, True))

## Running a classification experiment

In [ ]:
# No derivatives
model = phd.sermo.AuditoryFeatures()
expt = phd.experiments.AuditoryFeaturesExperiment(model, phones=phd.timit.TIMIT.phones)
expt.seed = 9
expt.timit.filefilt.region = 8
expt.cache_key

In [ ]:
# With 2 derivatives
model = phd.sermo.AuditoryFeatures()
model.add_derivative(klass="FeedforwardDeriv")
model.add_derivative(klass="FeedforwardDeriv")
expt = phd.experiments.AuditoryFeaturesExperiment(model, seed=0, phones=phd.timit.TIMIT.consonants)
expt.timit.filefilt.region = 1
result = expt.run()
print "Accuracy with MFCC:", result.mfcc_acc
print "Accuracy with NCC:", result.ncc_acc

In [ ]:
# With 1 derivative and z-score
model = phd.sermo.AuditoryFeatures()
model.add_derivative(klass="FeedforwardDeriv")
expt = phd.experiments.AuditoryFeaturesExperiment(model, seed=0, phones=phd.timit.TIMIT.consonants)
expt.zscore = True
expt.timit.filefilt.region = 1
result = expt.run()
print "Accuracy with MFCC:", result.mfcc_acc
print "Accuracy with NCC:", result.ncc_acc

## Plotting experimental results

In [ ]:
# Varying whether we're z-scoring
# phd.experiments.AuditoryFeaturesResult.subdir = 'ncc_12-03'
df = phd.analysis.load_results(phd.experiments.AuditoryFeaturesResult, ['zscore', 'phones'])

# Consonant accuracy by zscore
plt.figure()
phd.plots.compare(df,
                  columns=['mfcc_train_acc', 'mfcc_test_acc', 'ncc_train_acc', 'ncc_test_acc'],
                  group_by='zscore',
                  filter_by=[('phones', 'consonants')],
                  x_keys=['MFCC training', 'MFCC testing', 'NCC training', 'NCC testing'],
                  x_label='Feature',
                  y_label='Consonant accuracy',
#                   plot_f=sns.barplot,
                  hue_order=['False', 'True'])
plt.xlabel("")

# Speed
plt.figure()
phd.plots.compare(df,
                  columns=['mfcc_time', 'ncc_time', 'mfcc_fit_time', 'ncc_fit_time'],
                  group_by='zscore',
                  x_keys=['MFCC generation', 'NCC generation', 'MFCC SVM fitting', 'NCC SVM fitting'],
                  x_label='Scenario',
                  y_label='Time (s)',
                  plot_f=sns.barplot,
                  hue_order=['False', 'True'])
plt.ylabel("Time (s)")
plt.xlabel("")
# phd.experiments.AuditoryFeaturesResult.subdir = 'ncc'

In [ ]:
# Varying the number of derivatives used
phd.experiments.AuditoryFeaturesResult.subdir = 'ncc_12-03'
df = phd.analysis.load_results(phd.experiments.AuditoryFeaturesResult, ['derivatives', 'phones'])

# Vowel accuracy by zscore
phd.plots.compare(df,
                  columns=['mfcc_train_acc', 'mfcc_test_acc', 'ncc_train_acc', 'ncc_test_acc'],
                  group_by='derivatives',
                  filter_by=[('phones', 'vowels')],
                  x_keys=['MFCC training', 'MFCC testing', 'NCC training', 'NCC testing'],
                  x_label='Feature',
                  y_label='Vowel accuracy',
                  plot_f=sns.barplot,
                  hue_order=['0', '1', '2'])
plt.xlabel("")

# Consonant accuracy by zscore
plt.figure()
phd.plots.compare(df,
                  columns=['mfcc_train_acc', 'mfcc_test_acc', 'ncc_train_acc', 'ncc_test_acc'],
                  group_by='derivatives',
                  filter_by=[('phones', 'consonants')],
                  x_keys=['MFCC training', 'MFCC testing', 'NCC training', 'NCC testing'],
                  x_label='Feature',
                  y_label='Consonant accuracy',
                  plot_f=sns.barplot,
                  hue_order=['0', '1', '2'])
plt.xlabel("")

# Speed
plt.figure()
phd.plots.compare(df,
                  columns=['mfcc_time', 'ncc_time', 'mfcc_fit_time', 'ncc_fit_time'],
                  group_by='derivatives',
                  x_keys=['MFCC generation', 'NCC generation', 'MFCC SVM fitting', 'NCC SVM fitting'],
                  x_label='Scenario',
                  y_label='Time (s)',
                  plot_f=sns.barplot,
                  hue_order=['0', '1', '2'])

plt.ylabel("Time (s)")
plt.xlabel("")
phd.experiments.AuditoryFeaturesResult.subdir = 'ncc'

In [ ]:
df = phd.analysis.load_results(phd.experiments.AuditoryFeaturesResult, ['periphery', 'phones'])

# Consonant accuracy by zscore
plt.figure()
phd.plots.compare(df,
                  columns=['ncc_train_acc', 'ncc_test_acc'],
                  relative_to=['mfcc_train_acc', 'mfcc_test_acc'],
                  group_by='periphery',
                  filter_by=[('phones', 'consonants')],
                  x_keys=['Training', 'Testing'],
                  x_label='Feature',
                  y_label='Consonant accuracy',
                  hue_order=[str(x) for x in [1, 2, 4, 8, 16, 32]])
plt.axhline(1, c='k', ls=':', lw=1)
plt.xlabel("")

plt.figure()
phd.plots.timeseries(df,
                     columns=['ncc_train_acc', 'ncc_test_acc'],
                     relative_to=['mfcc_train_acc', 'mfcc_test_acc'],
                     group_by='periphery',
                     filter_by=[('phones', 'consonants')],
                     x_keys=['Training', 'Testing'],
                     x_label='Feature',
                     y_label='Consonant accuracy')
plt.axhline(1, c='k', ls=':', lw=1)
plt.xlabel("")

In [ ]:
df = phd.analysis.load_results(phd.experiments.AuditoryFeaturesResult, ['feature', 'phones'])

# Consonant accuracy by zscore
plt.figure()
phd.plots.compare(df,
                  columns=['ncc_train_acc', 'ncc_test_acc'],
                  relative_to=['mfcc_train_acc', 'mfcc_test_acc'],
                  group_by='feature',
                  filter_by=[('phones', 'consonants')],
                  x_keys=['Training', 'Testing'],
                  x_label='Feature',
                  y_label='Consonant accuracy',
                  hue_order=[str(x) for x in [1, 2, 4, 8, 12, 16, 32, 64]])
plt.axhline(1, c='k', ls=':', lw=1)
plt.xlabel("")

plt.figure()
phd.plots.timeseries(df,
                     columns=['ncc_train_acc', 'ncc_test_acc'],
                     relative_to=['mfcc_train_acc', 'mfcc_test_acc'],
                     group_by='feature',
                     filter_by=[('phones', 'consonants')],
                     x_keys=['Training', 'Testing'],
                     x_label='Feature',
                     y_label='Consonant accuracy')
plt.axhline(1, c='k', ls=':', lw=1)
plt.xlabel("")

# Speed
plt.figure()
phd.plots.compare(df,
                  columns=['ncc_time', 'ncc_fit_time'],
                  group_by='feature',
                  x_keys=['NCC generation', 'NCC SVM fitting'],
                  x_label='Scenario',
                  y_label='Time (s)',
                  plot_f=sns.barplot,
                  hue_order=[str(x) for x in [1, 2, 4, 8, 12, 16, 32, 64]])

plt.ylabel("Time (s)")
plt.xlabel("")

In [ ]:
df = phd.analysis.load_results(phd.experiments.AuditoryFeaturesResult, ['dt', 'phones'])

# Consonant accuracy
plt.figure()
phd.plots.compare(df,
                  columns=['mfcc_train_acc', 'mfcc_test_acc'],
                  group_by='dt',
                  filter_by=[('phones', 'consonants')],
                  x_keys=['MFCC training', 'MFCC testing'],
                  x_label='Feature',
                  y_label='Consonant accuracy',
                  plot_f=sns.barplot,
                  hue_order=["0.001000", "0.005000", "0.010000"])
plt.xlabel("")

In [ ]:
df = phd.analysis.load_results(phd.experiments.AuditoryFeaturesResult, ['phones'])

# Consonant accuracy
plt.figure()
phd.plots.compare(df,
                  columns=['ncc_train_acc', 'ncc_test_acc'],
                  relative_to=['mfcc_train_acc', 'mfcc_test_acc'],
                  group_by='phones',
                  x_keys=['Training', 'Testing'],
                  x_label='Feature',
                  y_label='Accuracy',
                  hue_order=["vowels", "consonants", "all"])
plt.axhline(1, c='k', ls=':', lw=1)
plt.xlabel("")

# Speed
plt.figure()
phd.plots.compare(df,
                  columns=['mfcc_time', 'ncc_time', 'mfcc_fit_time', 'ncc_fit_time'],
                  group_by='phones',
                  x_keys=['MFCC generation', 'NCC generation', 'MFCC SVM fitting', 'NCC SVM fitting'],
                  x_label='Scenario',
                  y_label='Time (s)',
                  plot_f=sns.barplot,
                  hue_order=["vowels", "consonants", "all"])
plt.ylabel("Time (s)")
plt.xlabel("")